In [1]:
!pip install optimum==1.16.1
!pip install transformers==4.36.0

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-02-16 16:01:44.185209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 16:01:44.185256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 16:01:44.186646: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_py

In [3]:
# a pre-trained model from HuggingFace
model_name = "bert-base-uncased"
onnx_directory = "bert-base-uncased_onnx"

In [4]:
# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", export=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
input_ids = tokenizer("I love PyTorch!", return_tensors="pt")
model(**input_ids)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0306,  0.2759]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [6]:
model_onnx = ORTModelForSequenceClassification.from_pretrained("bert-base-uncased", export=True)
model_onnx(**input_ids)

Framework not specified. Using pt to export to ONNX.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 2.2.0+cu121
Overriding 1 configuration item(s)
	- use_cache -> False


SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1623, -0.0475]]), hidden_states=None, attentions=None)

In [7]:
model_onnx.save_pretrained(onnx_directory)
tokenizer.save_pretrained(onnx_directory)

('bert-base-uncased_onnx/tokenizer_config.json',
 'bert-base-uncased_onnx/special_tokens_map.json',
 'bert-base-uncased_onnx/vocab.txt',
 'bert-base-uncased_onnx/added_tokens.json',
 'bert-base-uncased_onnx/tokenizer.json')

In [8]:
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from optimum.onnxruntime import ORTQuantizer

In [9]:
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)
quantizer = ORTQuantizer.from_pretrained(model_onnx)
quantizer.quantize(save_dir=onnx_directory, quantization_config=qconfig)

Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/s8, channel-wise: False)
Quantizing model...
Saving quantized model at: bert-base-uncased_onnx (external data format: False)
Configuration saved in bert-base-uncased_onnx/ort_config.json


PosixPath('bert-base-uncased_onnx')

In [10]:
model_quantized = ORTModelForSequenceClassification.from_pretrained(onnx_directory, file_name="model.onnx")
model_quantized(**input_ids)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1623, -0.0475]]), hidden_states=None, attentions=None)